In [7]:
import zipfile
import re

def to_csvstr(line: str, starts: list[int]):
    return (
        '",'.join(
            ['"' + line[start:stop].strip() for start, stop in zip(starts, starts[1:])]
        )
        + '"\n'
    )


with zipfile.ZipFile("players_list.zip") as z:
    with z.open("players_list_foa.txt", "r") as file:
        with open("inbetween.csv", "+a") as out_file:
            header = (
                file.readline().decode("utf-8").replace("ID Number", "IDNumber")
            )  # sigh
            starts = [m.span()[0] for m in re.finditer(r"([\w-]+)", header)]
            out_file.write(to_csvstr(header, starts))
            for line in file:
                decoded = line.decode("utf-8")
                out_file.write(to_csvstr(decoded, starts))

In [10]:
import pandas as pd

REPLACE_SCHEMA = {
    "IDNumber": "fideId",
    "Name": "name",
    "Tit": "title",
    "SRtng": "standardRating",
    "RRtng": "rapidRating",
    "BRtng": "blitzRating",
}
df = pd.read_csv("inbetween.csv", dtype_backend="pyarrow")
df.drop(columns=list(set(df.columns).difference(REPLACE_SCHEMA.keys())), inplace=True)
df.rename(columns=REPLACE_SCHEMA, inplace=True)
df.to_csv('fide.csv', index=False)